In [1]:
import time
import pandas as pd
import os
os.chdir('..')
import joblib

In [2]:
# feature columns
X_cols = [
    "hour","dayofweek","is_weekend","month",
    # "lag_24","rolling_24",
    # "electricity_current",
    "airTemperature", "dewTemperature", "windSpeed",
    # "temp_lag_1h","dewTemperature_lag_1h", "windSpeed_lag_1h",
    # "sqft", 
    "sqm", "primaryspaceusage", "site_id", "building_id",
    "Chilledwater", "Hotwater"
]

In [3]:
model_dir = "models_1578_csv"
os.makedirs(model_dir, exist_ok=True)

In [4]:
def load_data(path):
    data_df = pd.read_csv(path)
    return data_df

# TRAINING


In [5]:

data_encoded = "data_1578_csv/train_encode.csv"
data_frame = load_data(data_encoded)

In [6]:
# data_frame[["primaryspaceusage", "site_id", "buiding_id"]].head()
data_frame

,timestamp,Electricity,airTemperature,dewTemperature,windSpeed,Chilledwater,Hotwater,hour,dayofweek,is_weekend,...,target_t+15,target_t+16,target_t+17,target_t+18,target_t+19,target_t+20,target_t+21,target_t+22,target_t+23,target_t+24
0,2016-01-01 00:00:00,25.18,4.070588,0.305882,2.476471,196.0366,0.0000,0,4,0,...,25.09,25.44,25.18,26.69,25.95,25.80,25.74,25.69,25.68,25.63
1,2016-01-01 01:00:00,25.02,4.305556,0.305556,2.211111,201.7258,0.0000,1,4,0,...,25.44,25.18,26.69,25.95,25.80,25.74,25.69,25.68,25.63,25.44
2,2016-01-01 02:00:00,24.89,3.655556,0.038889,1.533333,201.4786,0.0000,2,4,0,...,25.18,26.69,25.95,25.80,25.74,25.69,25.68,25.63,25.44,25.57
3,2016-01-01 03:00:00,24.53,3.477778,-0.294444,2.183333,198.3866,0.0000,3,4,0,...,26.69,25.95,25.80,25.74,25.69,25.68,25.63,25.44,25.57,25.53
4,2016-01-01 04:00:00,25.09,3.488889,-0.527778,2.383333,199.9945,0.0000,4,4,0,...,25.95,25.80,25.74,25.69,25.68,25.63,25.44,25.57,25.53,25.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236496,2017-08-29 19:00:00,463.01,21.694118,14.664706,3.870588,5831.0038,2576.7095,19,1,0,...,492.37,511.68,506.44,509.95,526.04,509.59,503.64,522.87,465.41,456.94
1236497,2017-08-29 20:00:00,470.84,20.331579,14.268421,2.573684,5915.4789,2576.7095,20,1,0,...,511.68,506.44,509.95,526.04,509.59,503.64,522.87,465.41,456.94,459.71
1236498,2017-08-29 21:00:00,454.01,19.784211,14.068421,3.173684,5384.8819,1717.8064,21,1,0,...,506.44,509.95,526.04,509.59,503.64,522.87,465.41,456.94,459.71,453.31
1236499,2017-08-29 22:00:00,440.17,19.147368,13.910526,3.026316,5424.0891,2576.7095,22,1,0,...,509.95,526.04,509.59,503.64,522.87,465.41,456.94,459.71,453.31,432.20


In [7]:
import time
import xgboost as xgb

forecast_horizon = 24
times = time.time()
for h in range(forecast_horizon):
    model = xgb.XGBRegressor(
        tree_method="hist",
        n_estimators=500,
        max_depth=10,
        learning_rate=0.01,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42
    )
    
    target_col = f"target_t+{h+1}"
    df_train = data_frame[X_cols + [target_col]].dropna()
    print(f"Training model for horizon {h+1}, training samples: {len(df_train)}")
    X_train = df_train[X_cols]
    y_train = df_train[target_col]
    
    model.fit(X_train, y_train)
    joblib.dump(model, f"{model_dir}/model_hour_{h+1}.pkl")
print("Training time:", time.time() - times)

Training model for horizon 1, training samples: 1236501
Training model for horizon 2, training samples: 1236501
Training model for horizon 3, training samples: 1236501
Training model for horizon 4, training samples: 1236501
Training model for horizon 5, training samples: 1236501
Training model for horizon 6, training samples: 1236501
Training model for horizon 7, training samples: 1236501
Training model for horizon 8, training samples: 1236501
Training model for horizon 9, training samples: 1236501
Training model for horizon 10, training samples: 1236501
Training model for horizon 11, training samples: 1236501
Training model for horizon 12, training samples: 1236501
Training model for horizon 13, training samples: 1236501
Training model for horizon 14, training samples: 1236501
Training model for horizon 15, training samples: 1236501
Training model for horizon 16, training samples: 1236501
Training model for horizon 17, training samples: 1236501
Training model for horizon 18, training 